<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/cnn_duplicated_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN)

### Import TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers, models
# from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
import os
import random
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow as tf
from tqdm import tqdm
# Allow image embeding in notebook
%matplotlib inline
import time

In [0]:
def display_images_and_labels(images, labels):
    """Display the first image of each label."""
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(12, 12, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image)
    plt.show()

# display_images_and_labels(images, labels)

In [0]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    start = time.time()
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    index = []
    c = 0
    for d in tqdm(directories):
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm") or f.endswith(".jpg")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
            index.append(int(c))
            c = c+1
    end = time.time()
    print(end - start)
    return images, labels, index 


# Load training and testing datasets.
# ROOT_PATH = "/traffic"
train_data_dir = r"/content/gdrive/My Drive/traffic/Training"
test_data_dir = r"/content/gdrive/My Drive/traffic/Testing"
# test_data_dir = os.path.join(ROOT_PATH, "datasets/BelgiumTS/Testing")

train_images, train_labels, train_index = load_data(train_data_dir)
test_images, test_labels, test_index = load_data(test_data_dir)

In [0]:
# !wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip -P "/content/gdrive/My Drive/traffic"
# !unzip '/content/gdrive/My Drive/traffic/BelgiumTSC_Testing.zip' -d "/content/gdrive/My Drive/traffic"

In [0]:
# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in train_images]
test_images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in test_images]

train_labels_np = np.array(train_labels)
test_labels_np = np.array(test_labels)
train_index_np = np.array(train_index)
test_index_np = np.array(test_index)
train_images32_np = np.array(train_images32)
test_images32_np = np.array(test_images32)

In [0]:
c=0
tr_images32 = []
vr_images32 = []
tr_labels = []
vr_labels = []
for i in range(62):
  # print (i, sum(train_labels_np==i))
  c = c +sum (train_labels_np==i)
  # print (train_index_np[train_labels_np==i])
  a = train_index_np[train_labels_np==i]
  np.random.shuffle(a)
  # print(a)
  split = int(sum(train_labels_np==i)/10)+1
  tr_idx = a[split:]
  vr_idx = a[:split]
  for i in tr_idx:
    tr_images32.append(train_images32[i])
    tr_labels.append(train_labels[i])
  for i in vr_idx:
    vr_images32.append(train_images32[i])
    vr_labels.append(train_labels[i])
    # print(split)
print(len(tr_images32))
print(len(vr_images32))
print(len(tr_labels))
print(len(vr_labels))

In [0]:
tr_images32_np = np.array(tr_images32)
vr_images32_np = np.array(vr_images32)
tr_labels_np = np.array(tr_labels)
vr_labels_np = np.array(vr_labels)
print(tr_images32_np.shape)
print(vr_images32_np.shape)
print(tr_labels_np.shape)
print(vr_labels_np.shape)

In [0]:
tr_index_np = np.arange(tr_labels_np.shape[0])
vr_index_np = np.arange(vr_labels_np.shape[0])

In [0]:
from numpy.random import randint
dupli_images32 = []
dupli_labels = []
for particular_label in range(62):
  values = randint(0, sum(tr_labels_np == particular_label), 500)
  # print(sum(tr_labels_np == particular_label))
  # print(values)
  for i in (tr_index_np[tr_labels_np == particular_label][values]):
    # tr_images32[tr_index_np[tr_labels_np == particular_label][values]]
    dupli_images32.append(tr_images32[i])
    dupli_labels.append(tr_labels[i])
print(len(dupli_images32))
print(len(dupli_labels))

In [0]:
dupli_images32_np = np.array(dupli_images32)
dupli_labels_np = np.array(dupli_labels)
print(dupli_images32_np.shape)
print(dupli_labels_np.shape)

In [0]:
display_images_and_labels(tr_images32, tr_labels)

In [0]:
display_images_and_labels(vr_images32, vr_labels)

In [0]:
display_images_and_labels(dupli_images32, list(dupli_labels_np))

### Augmentation

In [0]:
folder_path = '/content/gdrive/My Drive/traffic/smote_augmented_data3'
path = '/content/gdrive/My Drive/traffic/smote_augmented_data3'
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
    
for i in range(62):
    path = '%s/%s' % (folder_path, i)
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [0]:
imgs_np_1d = np.array([(np.array(xi)).ravel() for xi in train_images32_np])

In [0]:
imgs_np_1d.shape

In [0]:
import skimage as sk
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import time
num_files_desired_per_label = 500
folder_path = '/content/gdrive/My Drive/traffic/smote_augmented_data3'
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=1000)
for particular_label in range(62):
    print (particular_label)
    num_clusters = 5
    if (sum(train_labels_np==particular_label)<3*num_clusters):
        num_clusters = int(sum(train_labels_np==particular_label)/4)    
    imgs_np_1d_particular_label = imgs_np_1d[train_labels_np == particular_label]
    index_particular_label = train_index_np[train_labels_np ==particular_label]
    tsne_results = tsne.fit_transform(imgs_np_1d_particular_label)
    kmeans = KMeans(n_clusters=num_clusters, max_iter = 500)
    kmeans_tsne = kmeans.fit_predict(tsne_results)
    
    num_vals_each_cluster = []
    for i in range(num_clusters):
        num_vals_each_cluster.append(sum(kmeans_tsne==i))
    actual_cluster = np.arange(num_clusters)
    actual_cluster = actual_cluster[np.array(num_vals_each_cluster)>1]
    num_files_desired_per_cluster = num_files_desired_per_label/num_clusters
    
    num_files_desired_per_cluster = num_files_desired_per_label/len(actual_cluster)
    
    for cluster_val in actual_cluster:

        num_files_present = len(index_particular_label[kmeans_tsne==cluster_val])        
        if(num_files_desired_per_cluster <= 1.5*num_files_present):
            file_idxs = np.arange(0, len(index_particular_label[kmeans_tsne==cluster_val]))
            np.random.shuffle(file_idxs)
            file_idxs = file_idxs[:int(min(num_files_desired_per_cluster,num_files_present))]
            for i in file_idxs:
                idx_label = (index_particular_label[kmeans_tsne==cluster_val][i])
                new_file_path = '%s/%s/augmented_image_%s.jpg' % (folder_path, particular_label, i)
                sk.io.imsave(new_file_path, train_images32_np[idx_label])

        else:
            dist_mat = np.ones((sum(kmeans_tsne==cluster_val),sum(kmeans_tsne==cluster_val)), dtype=np.float64)
            for i in range(sum(kmeans_tsne==cluster_val)):
                for j in range(i,sum(kmeans_tsne==cluster_val)):
                    a = tsne_results[kmeans_tsne==cluster_val][i]
                    b = tsne_results[kmeans_tsne==cluster_val][j]
                    dist = np.linalg.norm(a-b)
                    dist_mat[i,j] = float("{0:.2f}".format(dist))
                    dist_mat[j,i] = float("{0:.2f}".format(dist))

            k_nn_val_exp = round((num_files_desired_per_cluster-num_files_present)/num_files_present)
            k_nn_val = k_nn_val_exp
            c=0
            if (num_files_present-1<k_nn_val):
                k_nn_val = num_files_present-1
            for looper in range(int(k_nn_val_exp/k_nn_val)): 
                for i in range(sum(kmeans_tsne==cluster_val)):
                    knn = index_particular_label[kmeans_tsne==cluster_val][dist_mat.argsort()[:,1:k_nn_val+1]]
                    orig_img = (train_images32_np[index_particular_label[kmeans_tsne==cluster_val][i]])
                    g = np.random.uniform(size=k_nn_val)
                    p = np.zeros((k_nn_val,32,32,3))
                    for j in range(k_nn_val):
                        p[j] = g[j]
                    out_img = orig_img + p * (train_images32_np[knn[i]] - orig_img)
                    for j in range(k_nn_val):
                        c+=1
                        new_file_path = '%s/%s/augmented_image_%s_%s.jpg' % (folder_path, particular_label, knn[i][j], c)
                        sk.io.imsave(new_file_path,out_img[j])

                for i in index_particular_label[kmeans_tsne==cluster_val]:
                    new_file_path = "%s/%s/augmented_image_%s.jpg" % (folder_path, particular_label, i)
                    try:
                        sk.io.imsave(new_file_path, train_images32_np[i])
                        break
                    except:
                        print("***************************************************************")
                        print("***************************************************************")
                        print("***************************************************************")
                        print("***************************************************************")
                        print("***************************************************************")

In [0]:
aug_data_dir = r"/content/gdrive/My Drive/traffic/smote_augmented_data3"

aug_images, aug_labels, aug_index = load_data(aug_data_dir)

In [0]:
aug_labels_np = np.array(aug_labels)
aug_images_np = np.array(aug_images)
# for i in range(62):
#   print(i, sum(aug_labels_np==i))

In [0]:
aug_images_np.shape

In [0]:
for i in range(62):
  print(i, sum(aug_labels_np==i))

### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [0]:
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']

# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i], cmap=plt.cm.binary)
#     # The CIFAR labels happen to be arrays, 
#     # which is why you need the extra index
#     plt.xlabel(class_names[train_labels[i][0]])
# plt.show()

### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to our first layer.



In [0]:
# model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
model = models.Sequential()
model.add(layers.ZeroPadding2D((1,1),input_shape=(32,32,3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.ZeroPadding2D((1,1)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.ZeroPadding2D((1,1)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.ZeroPadding2D((1,1)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.ZeroPadding2D((1,1)))
# model.add(layers.MaxPooling2D((2, 2)))

Let's display the architecture of our model so far.

In [0]:
model.summary()

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically,  as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, you will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(62, activation='softmax'))

Here's the complete architecture of our model.

In [0]:
model.summary()

As you can see, our (3, 3, 64) outputs were flattened into vectors of shape (576) before going through two Dense layers.

### Compile and train the model

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# history = model.fit(aug_images_np, aug_labels_np, epochs=100, batch_size=128,
#                     validation_data=(test_images32_np,  test_labels_np))

history = model.fit(dupli_images32_np, dupli_labels_np, epochs=10, batch_size=64,
                    validation_data=(vr_images32_np, vr_labels_np))

In [0]:
for i in range(62):
  print (i, sum(aug_labels_np==i))
# (aug_labels_np.shape)

In [0]:
display_images_and_labels(aug_images_np, list(aug_labels_np))

In [0]:
display_images_and_labels(train_images, train_labels)

### Evaluate the model

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images32_np,  test_labels_np, verbose=2)

In [0]:
(model.predict(test_images32_np))[0]

In [0]:
test_pred = np.argsort(model.predict(test_images32_np))[:,61]
test_pred.shape

In [0]:
conf_arr = tf.math.confusion_matrix(
    test_labels_np,
    test_pred,
    num_classes=62,
    weights=None,
    dtype=tf.dtypes.int32,
    name=None
)

In [0]:
np.savetxt('2darray.csv', conf_arr, delimiter=',', fmt='%d')

In [0]:
print(test_acc)

Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).